# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/cities.csv"
df= pd.read_csv(output_data_file)
df.count()
df.describe()


,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
count,583.000000,583.000000,583.000000,583.00000,583.000000,583.000000
mean,20.524803,16.692178,67.091389,68.96741,50.348199,8.335232
std,33.200902,91.092362,14.690840,22.01079,38.522192,5.460822
min,-54.800000,-179.170000,28.400000,3.00000,0.000000,0.270000
25%,-6.560000,-65.515000,56.300000,56.50000,6.000000,4.285000
50%,23.450000,18.730000,68.000000,74.00000,53.000000,7.200000
75%,47.920000,100.345000,78.950000,86.50000,90.000000,11.410000
max,78.220000,179.320000,105.150000,100.00000,100.000000,31.340000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = df[["Latitude","Longitude"]]
Humidity = df["Humidity (%)"]



In [4]:
# Plot Heatmap
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout) 
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df.count()


City                583
Country             577
Latitude            583
Longitude           583
Temperature (F)     583
Humidity (%)        583
Cloudiness (%)      583
Wind Speed (mph)    583
dtype: int64

In [6]:
df_1=df
df_1=df_1.drop(df_1[df_1["Temperature (F)"] > 80].index) 
df_1=df_1.drop(df_1[df_1["Temperature (F)"] < 70].index)
df_1=df_1.drop(df_1[df_1["Cloudiness (%)"] != 0].index)
df_1=df_1.drop(df_1[df_1["Wind Speed (mph)"] > 10].index)

df_1.head()
df_1.count()





City                11
Country             11
Latitude            11
Longitude           11
Temperature (F)     11
Humidity (%)        11
Cloudiness (%)      11
Wind Speed (mph)    11
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = df_1
hotel_df['Hotel Name'] = hotel_df['Cloudiness (%)']
df_1


,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
73,São Filipe,CV,14.90,-24.50,75.11,82,0,9.91,0
131,Avarua,CK,-21.21,-159.78,73.40,64,0,6.93,0
383,Holiday,US,28.19,-82.74,75.65,89,0,7.43,0
415,Najrān,SA,17.49,44.13,75.20,23,0,5.50,0
468,Manoel Urbano,BR,-8.84,-69.26,71.71,78,0,1.16,0
480,Sumbe,AO,-11.21,13.84,70.75,74,0,4.32,0
489,Baghdad,IQ,33.34,44.40,78.80,50,0,8.05,0
513,Mossamedes,AO,-15.20,12.15,70.74,54,0,5.26,0
524,Penalva,BR,-3.29,-45.17,72.73,96,0,1.14,0
551,Lázaro Cárdenas,MX,17.96,-102.20,79.63,80,0,3.58,0


In [8]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    coordinates = str(hotel_df.loc[index,'Latitude']) +',' + str(hotel_df.loc[index,'Longitude'])
    params = {
        "key": g_key,
        "location": coordinates,
        "radius": 5000,
        "type": "hotel"
        }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
73,São Filipe,CV,14.90,-24.50,75.11,82,0,9.91,Sao Filipe
131,Avarua,CK,-21.21,-159.78,73.40,64,0,6.93,Avarua District
383,Holiday,US,28.19,-82.74,75.65,89,0,7.43,Tarpon Springs
415,Najrān,SA,17.49,44.13,75.20,23,0,5.50,Najran
468,Manoel Urbano,BR,-8.84,-69.26,71.71,78,0,1.16,Manoel Urbano
480,Sumbe,AO,-11.21,13.84,70.75,74,0,4.32,Sumbe
489,Baghdad,IQ,33.34,44.40,78.80,50,0,8.05,Baghdad
513,Mossamedes,AO,-15.20,12.15,70.74,54,0,5.26,Namibe
524,Penalva,BR,-3.29,-45.17,72.73,96,0,1.14,Penalva
551,Lázaro Cárdenas,MX,17.96,-102.20,79.63,80,0,3.58,Lázaro Cárdenas


In [9]:
# NOTE: Do not change any of the code in this cell
narrowed_city_df = hotel_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]



In [18]:
# Add marker layer ontop of heat map


# Add marker layer ontop of heat map
# Plot Heatmap
df= pd.read_csv(output_data_file)
locations = df[["Latitude", "Longitude"]]
Humidity = df["Humidity (%)"]

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig


# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))